In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
df_train = pd.read_csv("train-data.tsv", sep='\t', header=None, names=['class', 'text'])
df_val = pd.read_csv("valid-data.tsv", sep='\t', header=None, names=['class', 'text'])
# header = None -> prevents first row into column name

print(df_train.shape)
print(df_val.shape)
df_train

In [ ]:
max_len = max(len(text.split()) for text in df_train["text"])
print(max_len)

In [ ]:
# calculate vocabulary size (number of unique tokens)

vocabulary_dict = {}
for messgae in df_train.text:
  for vocabulary in messgae.split():
    if vocabulary not in vocabulary_dict:
      vocabulary_dict[vocabulary] = 1
    else:
      vocabulary_dict[vocabulary] += 1

max_features = len(vocabulary_dict)

print(max_features)

In [ ]:
# handle categorical data
# cat.codes return Series of codes and index
y_train = df_train['class'].astype('category').cat.codes
y_train = np.array(y_train)

y_val = df_val['class'].astype('category').cat.codes
y_val = np.array(y_val)

X_train = df_train.text
X_val = df_val.text

In [ ]:
# inspect data with bar chart

bar = df_train['class'].value_counts()
plt.bar(bar.index, bar)
plt.xlabel('class')
plt.title('Number of ham and spam messages')

**Text preprocessing - One Hot Encoding** from keras

In [ ]:
from keras.preprocessing.text import one_hot

encoded_train = [one_hot(text, max_features) for text in df_train.text]
encoded_val = [one_hot(text, max_features) for text in df_val.text]

# padding to the max length
X_train = pad_sequences(encoded_train, maxlen=max_len, padding='post')
X_val = pad_sequences(encoded_val, maxlen=max_len, padding='post')

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, LSTM, Embedding, Input, Dropout

batch_size = 32

model = Sequential()

# Specify the maximum input length as Input layer ('input_length' doesn't work as an attribute in the Embedding layer)
model.add(Input(shape = [max_len]))

model.add(Embedding(max_features, 100))

model.add(Dense(32, activation='relu'))

# We flatten the 3D tensor of embeddings into a 2D tensor
model.add(Flatten())

#model.add(Dropout(0.5))

# Add classifier on top
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val))

In [ ]:
# Visualize training curve (loss)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  encoded = [one_hot(pred_text, max_features)]
  padded = pad_sequences(encoded, maxlen=max_len, padding='post')

  class_dict = {
      0 : "ham",
      1 : "spam"
      }

  prediction = [model.predict(padded)[0][0], class_dict[np.round(model.predict(padded)[0][0])]]

  return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
